<a href="https://colab.research.google.com/github/rahulprasanth487/Semantic-Kernal-and-GraphRAG/blob/main/GraphRAG_Wiki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install --upgrade --quiet  langchain langchain-community langchain-groq neo4j
!pip install --upgrade --quiet  langchain-google-genai
!pip install --upgrade --quiet  langchain-openai langchain-experimental wikipedia tiktoken yfiles_jupyter_graphs

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.2/207.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.5/373.5 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 64.5 MB/s eta 0:00:00


In [5]:

NEO4J_URI="neo4j+s://13f6c03c.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="RZbRCl0pUoRTqPO0MoWYQN6PoMBdCbGUp_10H8Ws6bM"


In [6]:
from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)


In [23]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts.prompt import PromptTemplate

In [54]:
from google.colab import userdata
LLAMA_API_KEY=userdata.get('LLAMA_API_KEY')

In [9]:
from typing import Tuple, List, Optional
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import ConfigurableField
from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase

In [10]:
import os

In [11]:
from langchain_community.vectorstores import Neo4jVector

In [33]:

os.environ["NEO4J_URI"] = NEO4J_URI
os.environ["NEO4J_USERNAME"] = NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"] = NEO4J_PASSWORD

In [13]:
from langchain_community.graphs import Neo4jGraph

In [65]:
graph = Neo4jGraph()

In [66]:
graph

In [16]:
from langchain.document_loaders import WikipediaLoader
raw_documents = WikipediaLoader(query="Elizabeth I").load()

/usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [17]:
len(raw_documents)

23

In [19]:
raw_documents[:3]

[Document(metadata={'title': 'Elizabeth I', 'summary': 'Elizabeth I (7 September 1533 – 24 March 1603) was Queen of England and Ireland from 17 November 1558 until her death in 1603. She was the last monarch of the House of Tudor.\nElizabeth was the only surviving child of Henry VIII and his second wife, Anne Boleyn. When Elizabeth was two years old, her parents\' marriage was annulled, her mother was executed, and Elizabeth was declared illegitimate. Henry restored her to the line of succession when she was 10, via the Third Succession Act 1543. After Henry\'s death in 1547, Elizabeth\'s younger half-brother Edward VI ruled until his own death in 1553, bequeathing the crown to a Protestant cousin, Lady Jane Grey, and ignoring the claims of his two half-sisters, the Catholic Mary and the younger Elizabeth, in spite of statutes to the contrary. Edward\'s will was set aside within weeks of his death and Mary became queen, deposing and executing Jane. During Mary\'s reign, Elizabeth was i

In [44]:
from langchain.text_splitter import TokenTextSplitter

text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)
documents = text_splitter.split_documents(raw_documents[:3])

In [55]:
LLAMA_API_KEY
os.environ["LLAMA_API_KEY"] = LLAMA_API_KEY

In [56]:
# from langchain_google_genai import ChatGoogleGenerativeAI

# llm = ChatGoogleGenerativeAI(api_key=userdata.get('GEMINI_API_KEY') ,model="gemini-pro",convert_system_message_to_human=True)
# response = llm.invoke("Explain Quantum Computing in 50 words?")
# print(response.content)

In [57]:
from langchain_groq import ChatGroq

llm=ChatGroq(groq_api_key=LLAMA_API_KEY,model_name="llama3-70b-8192")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7e3740218460>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7e3740219120>, model_name='llama3-70b-8192', groq_api_key=SecretStr('**********'))

In [58]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
llm_transformer = LLMGraphTransformer(llm=llm)

In [59]:
llm_transformer

In [60]:
graph_documents = llm_transformer.convert_to_graph_documents(documents)

In [61]:
graph_documents[0].nodes

[Node(id='Elizabeth I', type='Person'),
 Node(id='Henry Viii', type='Person'),
 Node(id='Anne Boleyn', type='Person'),
 Node(id='Edward Vi', type='Person'),
 Node(id='Lady Jane Grey', type='Person'),
 Node(id='Mary', type='Person'),
 Node(id='William Cecil', type='Person'),
 Node(id='James Vi Of Scotland', type='Person'),
 Node(id='Mary, Queen Of Scots', type='Person')]

In [62]:
graph_documents[0].relationships

[Relationship(source=Node(id='Elizabeth I', type='Person'), target=Node(id='Henry Viii', type='Person'), type='CHILD'),
 Relationship(source=Node(id='Elizabeth I', type='Person'), target=Node(id='Anne Boleyn', type='Person'), type='CHILD'),
 Relationship(source=Node(id='Edward Vi', type='Person'), target=Node(id='Henry Viii', type='Person'), type='CHILD'),
 Relationship(source=Node(id='Mary', type='Person'), target=Node(id='Henry Viii', type='Person'), type='CHILD'),
 Relationship(source=Node(id='Elizabeth I', type='Person'), target=Node(id='William Cecil', type='Person'), type='ADVISOR'),
 Relationship(source=Node(id='James Vi Of Scotland', type='Person'), target=Node(id='Elizabeth I', type='Person'), type='SUCCESSOR')]

In [63]:
graph_documents

[GraphDocument(nodes=[Node(id='Elizabeth I', type='Person'), Node(id='Henry Viii', type='Person'), Node(id='Anne Boleyn', type='Person'), Node(id='Edward Vi', type='Person'), Node(id='Lady Jane Grey', type='Person'), Node(id='Mary', type='Person'), Node(id='William Cecil', type='Person'), Node(id='James Vi Of Scotland', type='Person'), Node(id='Mary, Queen Of Scots', type='Person')], relationships=[Relationship(source=Node(id='Elizabeth I', type='Person'), target=Node(id='Henry Viii', type='Person'), type='CHILD'), Relationship(source=Node(id='Elizabeth I', type='Person'), target=Node(id='Anne Boleyn', type='Person'), type='CHILD'), Relationship(source=Node(id='Edward Vi', type='Person'), target=Node(id='Henry Viii', type='Person'), type='CHILD'), Relationship(source=Node(id='Mary', type='Person'), target=Node(id='Henry Viii', type='Person'), type='CHILD'), Relationship(source=Node(id='Elizabeth I', type='Person'), target=Node(id='William Cecil', type='Person'), type='ADVISOR'), Relati

In [67]:
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

In [71]:
#directly show the graph resulting from the given Cypher query
default_cypher = "MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t LIMIT 50"

In [68]:
from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase

In [69]:
try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass

In [72]:
def showGraph(cypher: str = default_cypher):
    # create a neo4j session to run queries
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))
    session = driver.session()
    widget = GraphWidget(graph = session.run(cypher).graph())
    widget.node_label_mapping = 'id'
    display(widget)
    return widget

In [73]:
showGraph()

GraphWidget(layout=Layout(height='800px', width='100%'))

GraphWidget(layout=Layout(height='800px', width='100%'))

In [94]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 2.7 MB/s eta 0:00:00


In [96]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Neo4jVector

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)

vector_index = Neo4jVector.from_existing_graph(
    embeddings,
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [97]:
vector_index

In [77]:
from langchain.chains import GraphCypherQAChain

In [87]:
# chain=GraphCypherQAChain.from_llm(llm=llm,graph=graph,verbose=True)
# chain

In [98]:
graph.query("CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR (e:__Entity__) ON EACH [e.id]")

[]

In [83]:
from langchain_core.pydantic_v1 import BaseModel, Field

class Entities(BaseModel):
    """Identifying information about entities."""

    names: List[str] = Field(
        ...,
        description="All the person, organization, or business entities that "
        "appear in the text",
    )


In [84]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

In [85]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting organization and person entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)

In [86]:
entity_chain = prompt | llm.with_structured_output(Entities)

In [91]:
entity_chain.invoke({"question": "Where was Amelia Earhart born?"}).names

['Amelia Earhart']

In [99]:
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars

In [100]:
def generate_full_text_query(input: str) -> str:
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()

In [101]:
# Fulltext index query
def structured_retriever(question: str) -> str:
    result = ""
    entities = entity_chain.invoke({"question": question})
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": generate_full_text_query(entity)},
        )
        result += "\n".join([el['output'] for el in response])
    return result

In [102]:
print(structured_retriever("Who is Elizabeth I?"))

Elizabeth I - CHILD -> Henry Viii
Elizabeth I - CHILD -> Anne Boleyn
Elizabeth I - ADVISOR -> William Cecil
James Vi Of Scotland - SUCCESSOR -> Elizabeth I
Queen Elizabeth I - FATHER -> Henry Viii
Queen Elizabeth I - OFFICIAL_LIMNER -> Nicholas Hilliard
Queen Elizabeth I - SERJEANT_PAINTER -> George Gower


In [103]:

def retriever(question: str):
    print(f"Search query: {question}")
    structured_data = structured_retriever(question)
    unstructured_data = [el.page_content for el in vector_index.similarity_search(question)]
    final_data = f"""Structured data:
{structured_data}
Unstructured data:
{"#Document ". join(unstructured_data)}
    """
    return final_data

In [105]:
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question,
in its original language.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""

In [106]:
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

In [107]:
def _format_chat_history(chat_history: List[Tuple[str, str]]) -> List:
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer

In [109]:
_search_query = RunnableBranch(
    # If input includes chat_history, we condense it with the follow-up question
    (
        RunnableLambda(lambda x: bool(x.get("chat_history"))).with_config(
            run_name="HasChatHistoryCheck"
        ),  # Condense follow-up question and chat into a standalone_question
        RunnablePassthrough.assign(
            chat_history=lambda x: _format_chat_history(x["chat_history"])
        )
        | CONDENSE_QUESTION_PROMPT
        | ChatGroq(groq_api_key=LLAMA_API_KEY,model_name="llama3-70b-8192")
        | StrOutputParser(),
    ),
    # Else, we have no chat history, so just pass through the question
    RunnableLambda(lambda x : x["question"]),
)

In [110]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
Use natural language and be concise.
Answer:"""

In [111]:
prompt = ChatPromptTemplate.from_template(template)

In [126]:
# chain = (
#     RunnableParallel(
#         {
#             "context": _search_query | retriever,
#             "question": RunnablePassthrough(),
#         }
#     )
#     | prompt
#     | llm
#     | StrOutputParser()
# )

chain = (
    RunnableParallel(
        {
            "context": _search_query | retriever,
            "question": RunnablePassthrough(),
        }
    )
    | prompt
    | llm
    | (lambda result: print(f"LLM Output: {result}") or result)  # Print LLM result
    | StrOutputParser()
)


In [127]:
chain.invoke({"question": "Which house did Elizabeth I belong to?"})

Search query: Which house did Elizabeth I belong to?


LLM Output: content='Elizabeth I belonged to the House of Tudor.' response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 1916, 'total_tokens': 1927, 'completion_time': 0.035873111, 'prompt_time': 0.129231749, 'queue_time': 0.004595739999999987, 'total_time': 0.16510486}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_87cbfbbc4d', 'finish_reason': 'stop', 'logprobs': None} id='run-7eef940d-466d-4806-bf28-11e40d057bd3-0' usage_metadata={'input_tokens': 1916, 'output_tokens': 11, 'total_tokens': 1927}


'Elizabeth I belonged to the House of Tudor.'

In [128]:
chain.invoke(
    {
        "question": "When was she born?",
        "chat_history": [("Which house did Elizabeth I belong to?", "House Of Tudor")],
    }
)

Search query: What is the birthdate of Elizabeth I?


LLM Output: content='Elizabeth I was born on September 7, 1533.' response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 1932, 'total_tokens': 1946, 'completion_time': 0.045054701, 'prompt_time': 0.119073648, 'queue_time': 0.006158771000000007, 'total_time': 0.164128349}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_c1a4bcec29', 'finish_reason': 'stop', 'logprobs': None} id='run-f049e27d-bfbe-439c-8769-5040ff3d8b8b-0' usage_metadata={'input_tokens': 1932, 'output_tokens': 14, 'total_tokens': 1946}


'Elizabeth I was born on September 7, 1533.'